
# Tutorial: Implementation with `cross_val_score`

Setelah memahami konsep K-Fold Cross-Validation, sekarang saatnya kita mendalami alat utama dari Scikit-learn untuk menjalankannya: `cross_val_score`.

Fungsi ini adalah "pisau Swiss Army" Anda untuk evaluasi model. Ia merangkum seluruh proses K-Fold (membagi data, melatih model K kali, mengevaluasi K kali) ke dalam satu baris kode yang bersih dan efisien.

Di notebook ini, kita akan membedah cara menggunakan `cross_val_score`, terutama fokus pada parameternya yang paling penting seperti `cv` dan `scoring`.




---
### 1. Tujuan Pembelajaran

Di akhir notebook ini, Anda akan dapat:

* Menggunakan fungsi `cross_val_score` untuk melakukan K-Fold Cross-Validation.
* Menjelaskan parameter-parameter utama: `estimator`, `X`, `y`, `cv`, dan `scoring`.
* Meminta metrik evaluasi yang berbeda (seperti MSE atau MAE) menggunakan parameter `scoring`.
* Memahami konvensi `neg_` (misalnya, `neg_mean_squared_error`) di Scikit-learn dan cara menginterpretasikannya.




---
### 2. Setup: Mempersiapkan Data dan Model

Kita akan melanjutkan dengan dataset California Housing dan model Regresi Linear untuk konsistensi.



In [1]:

import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

# Muat dataset
housing = fetch_california_housing()
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = pd.Series(housing.target, name='MedHouseVal')

# Inisialisasi model yang akan kita evaluasi
model = LinearRegression()



/usr/local/lib/python3.11/site-packages/sklearn/datasets/_base.py:1519: UserWarning: Retry downloading from url: https://ndownloader.figshare.com/files/5976036
  warnings.warn(f"Retry downloading from url: {remote.url}")


HTTPError: HTTP Error 403: Forbidden


---
### 3. Sintaks Dasar `cross_val_score`

Mari kita lihat kembali sintaks dasarnya dan bedah setiap komponennya.

```python
cross_val_score(estimator, X, y, cv, scoring)
```

* `estimator`: Objek model Scikit-learn yang ingin Anda evaluasi (misalnya, `LinearRegression()`).
* `X`: Fitur-fitur dari **keseluruhan dataset Anda**.
* `y`: Target dari **keseluruhan dataset Anda**.
* `cv`: Jumlah *folds* (lipatan) yang diinginkan. Angka 5 atau 10 adalah pilihan umum.
* `scoring`: String yang menentukan metrik evaluasi apa yang harus dihitung.



In [ ]:
score_r2 = cross_val_score(model,X,y,cv=10)

print(f"hasil 10-fold CV (R2): \n{score_r2}")
print(f"\nrata-rata r2: {np.mean(score_r2):.4f} (+/- {np.std(score_r2):.4f})")


hasil 10-fold CV (R2): 
[0.48254494 0.61416063 0.42274892 0.48178521 0.55705986 0.5412919
 0.47496038 0.45844938 0.48177943 0.59528796]

rata-rata r2: 0.5110 (+/- 0.0593)



---
### 4. Mendalami Parameter `scoring`: Meminta Metrik yang Berbeda

Bagaimana jika kita tidak tertarik pada R² dan lebih peduli pada **Root Mean Squared Error (RMSE)**? Di sinilah parameter `scoring` menjadi sangat penting.

Scikit-learn memiliki konvensi penting: **semua metrik `scoring` dirancang agar "semakin tinggi semakin baik"**.

Ini tidak masalah untuk metrik seperti R², di mana nilai yang lebih tinggi memang lebih baik. Tetapi untuk metrik *error* (seperti MAE, MSE, RMSE), nilai yang lebih **rendah** justru lebih baik.

Untuk mengatasi ini, Scikit-learn menyediakan *scorer* untuk *error* dalam bentuk **negatif**. Misalnya, untuk menghitung MSE, kita harus meminta `'neg_mean_squared_error'`.



In [ ]:
score_mse = cross_val_score(model,X,y,cv=10,scoring="neg_mean_squared_error")

print(f"hasil 10-fold CV (neg_mse): \n{score_mse}")
print(f"\nrata-rata neg_mse: {np.mean(score_mse):.4f} (+/- {np.std(score_mse):.4f})")

hasil 10-fold CV (neg_mse): 
[-0.48922052 -0.43335865 -0.8864377  -0.39091641 -0.7479731  -0.52980278
 -0.28798456 -0.77326441 -0.64305557 -0.3275106 ]

rata-rata neg_mse: -0.5510 (+/- 0.1929)



#### Menginterpretasikan Hasil Negatif

Seperti yang Anda lihat, semua skor MSE bernilai negatif. Untuk mendapatkan nilai MSE yang sebenarnya, kita hanya perlu mengalikannya dengan -1.

Dan untuk mendapatkan RMSE, kita melakukan langkah tambahan: mengakarkannya.



In [ ]:
positive_mse = -score_mse
print(f"hasil 10-fold CV (mse positif):\n{positive_mse}")

mean_mse = np.mean(positive_mse)
std_mse = np.std(positive_mse)

print(f"\nrata-rata mse: {mean_mse:.4f} (+/- {std_mse:.4f})")

rmse_scores = np.sqrt(positive_mse)
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)
print(f"\nhasil 10-fold CV (rmse positif):\n{rmse_scores}")
print(f"\nrata-rata rmse: {mean_rmse:.4f} (+/- {std_rmse:.4f})")

hasil 10-fold CV (mse positif):
[0.48922052 0.43335865 0.8864377  0.39091641 0.7479731  0.52980278
 0.28798456 0.77326441 0.64305557 0.3275106 ]

rata-rata mse: 0.5510 (+/- 0.1929)

hasil 10-fold CV (rmse positif):
[0.69944301 0.65829982 0.94150821 0.62523308 0.86485438 0.72787552
 0.53664192 0.87935454 0.80190746 0.57228542]

rata-rata rmse: 0.7307 (+/- 0.1303)



Sekarang kita memiliki estimasi yang andal untuk RMSE model kita. Kita bisa mengatakan, "Model kami diperkirakan memiliki kesalahan prediksi rata-rata sekitar 0.73 (dalam unit target) dengan variasi sekitar 0.13."

#### Tabel `scoring` Umum untuk Regresi

| String `scoring` | Metrik yang Dihitung |
| :--- | :--- |
| `'r2'` | R-squared |
| `'neg_mean_squared_error'` | Mean Squared Error (negatif) |
| `'neg_root_mean_squared_error'`| Root Mean Squared Error (negatif) |
| `'neg_mean_absolute_error'` | Mean Absolute Error (negatif) |




---
### 5. Kesimpulan

* `cross_val_score` adalah fungsi utama di Scikit-learn untuk melakukan evaluasi model yang andal.
* Gunakan parameter `cv` untuk menentukan jumlah *folds* (lipatan).
* Gunakan parameter `scoring` untuk menentukan metrik evaluasi yang Anda inginkan.
* Ingatlah konvensi Scikit-learn: metrik *error* (MAE, MSE, RMSE) harus diminta dalam bentuk negatifnya (misalnya, `'neg_mean_absolute_error'`) dan kemudian diubah kembali menjadi positif untuk interpretasi.
